In [1]:
import pandas as pd
import numpy as np
# для парсинга
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
def wall_parser():
    token = ''
    api = 'https://api.vk.com/method/wall.get'
    version = 5.92
    domain = 'knife.media'
    offset = 0
    count = 100
    all_posts = []

    while offset < 9000:
        response = requests.get(api,
                                params = {'access_token': token,
                                          'v': version,
                                          'domain': domain,
                                          'count': count,
                                          'offset': offset}
                               )
        data = response.json()['response']['items']
        offset += count
        all_posts.extend(data)
    return all_posts

In [3]:
def filter_inf():
    filter_data_list = []
    all_posts = wall_parser()
    for post in all_posts:
        filter_data_list.append([post['id'],
                                 post['date'],
                                 post['likes']['count'],
                                 post['comments']['count'],
                                 post['reposts']['count'],
                                 post['views']['count'],
                                 post['text']])
    return filter_data_list

In [4]:
def get_url(x):
    pattern = r'(https://kn.+)|(http://knf.+)'
    try:
        return re.search(pattern, x)[0]
    except:
        return ('false')

In [5]:
# функция парсинга тегов
def get_tags(link):
    tags_page = []
    try:
        html = urlopen(link).read().decode('utf-8')
        soup = BeautifulSoup(html,'html.parser')
        tags = soup.find_all(rel="tag")
        for tag in tags:
            tags_page.append(tag.text)
    except:
        return None
    if len(tags_page) == 0:
        return None
    return tags_page

In [6]:
# формируем датафрейм

In [7]:
columns = ['id', 'time', 'likes', 'comments', 'reposts', 'views', 'text']
df = pd.DataFrame(filter_inf(), columns = columns)

In [8]:
# рассчитываем метрику, по которой в дальнейшем будем оценивать качество постов
df['ctr'] = (df.likes / df.views * 100).round(2)

In [9]:
# парсим ссылки из заголовков постов
df['url'] = df.text.apply(get_url)

In [10]:
df['year'] = pd.to_datetime(df.time, unit='s').dt.to_period('Y')
df['month'] = pd.to_datetime(df.time, unit='s').dt.to_period('M')
df['day'] = pd.to_datetime(df.time, unit='s').dt.to_period('D')

In [11]:
df.head()

,id,time,likes,comments,reposts,views,text,ctr,url,year,month,day
0,320422,1605261747,51,8,1,17555,"«Нож» ищет новостника — быстрого, сообразитель...",0.29,https://knf.md/news/,2020,2020-11,2020-11-13
1,320900,1605604268,57,3,3,7159,Мужчина ухаживает за животными уже 25 лет. Де...,0.80,https://knife.media/raccoon-whisperer/,2020,2020-11,2020-11-17
2,320899,1605604203,33,1,5,7926,"Знаете, как добывали подноготную правду, зачем...",0.42,https://knife.media/origin-of-idioms/,2020,2020-11,2020-11-17
3,320894,1605600004,22,0,2,7755,"От проектов, помогающих детям-сиротам, до IT-с...",0.28,https://knife.media/edu-projects/,2020,2020-11,2020-11-17
4,320828,1605546303,106,0,13,14448,"Сказки братьев Гримм знают и любят и все, хотя...",0.73,https://knife.media/bratya-grimm/,2020,2020-11,2020-11-16


In [12]:
filtered_df = df.query("url != 'false' and day >= '2018-01-01'")

In [13]:
filtered_df.shape

(7869, 12)

In [14]:
filtered_df['tags'] = filtered_df.url.apply(get_tags)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
filtered_df.isna().sum()

id            0
time          0
likes         0
comments      0
reposts       0
views         0
text          0
ctr           0
url           0
year          0
month         0
day           0
tags        243
dtype: int64

In [16]:
filtered_df.dropna(inplace=True)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
filtered_df.to_csv('1_filtered_df.csv', index=False)